# SQL Query

*Easily search through the lighting database with this script*

## Import of libraries needed for sql search, dataframe use, and html plot

In [1]:
import pyodbc
import pandas as pd

from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Category20
from bokeh.io import output_notebook
from bokeh.models import HoverTool 
import itertools

import numpy as np

## Database Connection
*Name of server, database, utilizes ad3 user and password from computer*

In [2]:
server = 'cfo-sql1' #name of the sql server 
database = 'Lighting' #name of the database in the sql server
username = '' #will use ad3 user and password for computer 
password =  ''

#Command for connecting to sql server using pyodbc connect
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';PORT=1443;DATABASE='+database+';UID='+username+';PWD='+ password) 
cursor = cnxn.cursor()

## Inputs: Building Name, Measure Type, Start Date, End Date, Frequency
*Building Example Inputs: 'Kemper', 'Ghausi', 'Meyer', 'Gourley', 'Hart'...*

*Measure Type Examples: 'KW', 'Daylight', 'Occupancy', 'Switchlock', 'Afterhours'*

In [3]:
buildings_input = input("Enter Building Name: ") 
meastype_input = input("Enter Measuring Type: ")
if (buildings_input == "") or (meastype_input == "") :
    print("Please type building name or measure type.")
    
split_buildings = buildings_input.split(' ')
split_meastype = meastype_input.split(' ')

Enter Building Name: Gourley
Enter Measuring Type: lkw


*Required Date Input Format (mm/dd/yyyy)*

In [4]:
start_date = input("Enter Start Date: ")
end_date = input("Enter End Date: ")

Enter Start Date: 01/01/2019
Enter End Date: 01/31/2019


*Frequency Examples ~ 'D': Daily, 'H': Hourly, '15min': Every 15 minutes (whatever #min desired)*

In [5]:
frequency = input("Enter frequency: ")

Enter frequency: 15min


# Collecting tables that meet sql search criteria

*Searches through database based on building name and measure type*

In [6]:
building_name = []
meastype = []

for name in split_buildings:
    for measure in split_meastype:
        building_name = name
        meastype = measure
        filtered_array = []
        read_sql = "SELECT * from Sys.Tables WHERE name like '%_" + meastype + "%' and name like '%" + building_name + "%'"

*Creates a dataframe with table names that meet criteria*

In [7]:
    with cursor.execute(read_sql):
        row = cursor.fetchone() #retrieves next row of query result set and returns single sequence
        while row:
            filtered_array.append({'table': row[0]})
            row = cursor.fetchone()
        
    #converting table into dataframe with table names of sql search that meets criteria
    filtered_dataframe = pd.DataFrame(filtered_array) 
    display(filtered_dataframe)

,table
0,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1102_LKW
1,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1125_LKW
2,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1127_LKW
3,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1129_LKW
4,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1130_LKW
5,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1130A_LKW
6,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1130B_LKW
7,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1131_LKW
8,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1133_LKW
9,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1135_LKW


*Searching specific date and frequency through filtered dataframe*

In [8]:
    timerange = pd.date_range(start = start_date, end = end_date, freq = frequency)
    final_dataframe = pd.DataFrame()

    neveroff = pd.DataFrame(columns=['Room', 'Average', 'Standard Div'])
    
    for column in filtered_dataframe['table']:
        building = []
        read_sql = "SELECT [TIMESTAMP],[VALUE] from [dbo].["+column+"] WHERE [TIMESTAMP] BETWEEN "  + "'" + start_date +"'" + " AND " + "'"+end_date+"'"
        with cursor.execute(read_sql):
            row = cursor.fetchone()
            while row:
                building.append({'date':row[0], column:row[1]})
                row = cursor.fetchone()
        building = pd.DataFrame(building)
        try: #when table column is empty
            building = building.set_index('date') #sets date column as index
            #filter to minute resolution
            building.index = building.index.map(lambda x: x.replace(second=0)) 
            building.index = building.index.map(lambda x: x.replace(microsecond=0))
            #filtering in case of duplicates
            building = building[~building.index.duplicated(keep='first')]
            
            #occupancy sensor query
            if meastype == 'OCCUPANCY':
                building = building.reindex(timerange)
            else:
                building = building.resample(frequency).backfill()
                
                
            final_dataframe = pd.concat([building, final_dataframe], axis=1)
            #print(column)
            
            #neveroff calculation
            averagevalue = building.mean(). iloc[0]
            maxvalue = building.max(). iloc[0]
            stdiv = np.std([averagevalue, maxvalue])
            if (averagevalue != 0) and (stdiv < .01):
                neveroff = neveroff.append({'Room':column, 'Average':averagevalue,'Standard Div':stdiv},ignore_index=True)
        except:
            pass
display(neveroff)

,Room,Average,Standard Div
0,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1102_LKW,0.007268,0.008866
1,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1145_LKW,0.087000,0.000000


*Turns Boolean into '1' for on and '0' for off*

In [9]:
    final_dataframe *=1

*Outputs to CSV file*

In [10]:
    final_dataframe.columns = sorted(final_dataframe.columns, key=lambda item: (int(item.partition(' ')[0]) if item[0].isdigit() else float('inf'), item))
    start_date = start_date.replace('/', '_')
    final_dataframe.to_csv(buildings_input+'_'+meastype_input+'_DATA_'+start_date+'.csv') #exports to csv and appears in file explorer
    
    display(final_dataframe)

,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1102_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1125_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1127_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1129_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1130A_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1130B_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1130_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1131_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1133_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1135_LKW,...,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1160C_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1160D_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1160_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1180_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1195_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1200_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1202_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1204_LKW,UCDAVIS_GOURLEYCLINICALCNTR_01_RM_1214_LKW,UCDAVIS_GOURLEYCLINICALCNTR_LKW
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,0.648,0.0,0.000,0.000,0.000,0.561,0.000,0.0,0.0,0.0,...,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.0
2019-01-01 00:15:00,0.648,0.0,0.000,0.000,0.000,0.561,0.000,0.0,0.0,0.0,...,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.0
2019-01-01 00:30:00,0.648,0.0,0.000,0.000,0.000,0.561,0.000,0.0,0.0,0.0,...,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.0
2019-01-01 00:45:00,0.648,0.0,0.000,0.000,0.000,0.561,0.000,0.0,0.0,0.0,...,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.0
2019-01-01 01:00:00,0.648,0.0,0.000,0.000,0.000,0.561,0.000,0.0,0.0,0.0,...,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.0
2019-01-01 01:15:00,0.648,0.0,0.000,0.000,0.000,0.561,0.000,0.0,0.0,0.0,...,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.0
2019-01-01 01:30:00,0.648,0.0,0.000,0.000,0.000,0.561,0.000,0.0,0.0,0.0,...,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.0
2019-01-01 01:45:00,0.648,0.0,0.000,0.000,0.000,0.561,0.000,0.0,0.0,0.0,...,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.0
2019-01-01 02:00:00,0.648,0.0,0.000,0.000,0.000,0.561,0.000,0.0,0.0,0.0,...,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.0


## Plots data on HTML file with Bokeh Plotting

*Asks user if they want to plot the data*

In [11]:
    plot_desired = input("Plot? 'Y' or 'N': ")
    
    if plot_desired == 'Y':
         def color_gen():
            for c in itertools.cycle(Category20[10]):
                yield c

Plot? 'Y' or 'N': Y


*Plot data*

In [15]:
        TOOLTIPS = [('date', ('datetime')), (meastype, '($y)')] #not displaying date
        graph = figure(plot_width = 1000, plot_height=800, x_axis_type = 'datetime', tooltips = TOOLTIPS, title = buildings_input+'_'+meastype_input+'_DATA_'+start_date)
        graph.xaxis.axis_label = 'Date'
        graph.yaxis.axis_label = meastype
    
        colors = color_gen()
        tags = final_dataframe.columns
        for tag in tags:
            color = next(colors)
            graph.circle(final_dataframe.index, final_dataframe[tag], size = 10, color = color, legend = tag)
            graph.line(final_dataframe.index, final_dataframe[tag], color = color, legend = tag)
            
        graph.legend.click_policy = 'hide'
        legend_position = 'bottom'

        #output_file(buildings_input+'_'+meastype_input+'_DATA_'+start_date+'.html')  
        
        output_notebook()
        show(graph)

Loading BokehJS ...